In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngetionConfig:
    root_dir:Path 
    local_data_file:Path
    unzip_dir:Path

In [5]:
from QuantumThreads.constants import *
from QuantumThreads.utils.common import read_yaml, create_directories, get_size

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        
    def get_data_ingetion_config(self)-> DataIngetionConfig:
        
        config=self.config.data_ingetion
        
        create_directories([config.root_dir])
        data_ingetion_config=DataIngetionConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingetion_config   

In [7]:
from urllib import request
from zipfile import ZipFile
from QuantumThreads.logging import logger
from QuantumThreads.utils.common import get_size

In [25]:
class DataTransformation:
    def __init__(self,
                 config:DataIngetionConfig):
        
        self.config=config
    
    def data_loader(self):
        image_dir=self.config.unzip_dir
        datagen = ImageDataGenerator(
            rescale=1./255,
            
        )    
        batch_size = 32
        train_data = tf.keras.preprocessing.image_dataset_from_directory(
            image_dir,
            image_size=(224, 224),
            batch_size=batch_size,
            label_mode='categorical', 
            validation_split=0.2,
            seed=42,
            subset='training',
            color_mode='grayscale'
        )
        
        self.train_data = train_data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return self.train_data
        
       

In [26]:
try:
    config=ConfigurationManager()
    data_ingetion_config=config.get_data_ingetion_config()
    data_ingetion=DataTransformation(data_ingetion_config)
    train_data=data_ingetion.data_loader()
except Exception as e:
    raise e

[2024-01-08 22:04:56,409 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-01-08 22:04:56,414 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-01-08 22:04:56,417 : INFO : common : created directory at: dataset/data_ingestion]
Found 13711 files belonging to 23 classes.
Using 10969 files for training.
